# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [23]:
## Importar Librerías
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [24]:
## Funciones Auxiliares
def get_star_rating(star_str):
    ratings = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}
    return ratings.get(star_str, 0)


In [25]:

def get_price(price_str):
    return float(price_str.replace('Â', '').replace('£', ''))

In [26]:
## Extraer Datos
def extract_books_from_page(soup):
    books = []
    articles = soup.find_all('article', class_='product_pod')
    for article in articles:
        title = article.h3.a['title']
        price = get_price(article.find('p', class_='price_color').text.strip())
        rating = get_star_rating(article.find('p', class_='star-rating')['class'][1])
        availability = article.find('p', class_='instock availability').text.strip()
        book_url = 'http://books.toscrape.com/catalogue/' + article.h3.a['href']
        
        books.append({
            "Title": title,
            "Price (£)": price,
            "Rating": rating,
            "Availability": availability,
            "Book URL": book_url
        })
    return books


In [27]:

## Extraer Información Detallada
def extract_book_details(book_url):
    response = requests.get(book_url)
    book_soup = BeautifulSoup(response.content, 'html.parser')
    
    try:
        genre = book_soup.find('ul', class_='breadcrumb').find_all('a')[2].text.strip()
    except:
        genre = "Unknown"
    
    try:
        upc = book_soup.find('th', text='UPC').find_next_sibling('td').text.strip()
    except:
        upc = "N/A"
    
    description_tag = book_soup.find('meta', {'name': 'description'})
    description = description_tag['content'].strip() if description_tag else "No description available"
    
    return genre, upc, description

In [28]:
## scrape_books
def scrape_books(min_rating, max_price):
    base_url = 'http://books.toscrape.com/'
    books_data = []
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
    
    page_num = 1
    max_pages = 50
    while page_num <= max_pages:
        try:
            url = f'{base_url}catalogue/page-{page_num}.html'
            response = requests.get(url, headers=headers, timeout=10)
            if response.status_code != 200:
                break
            
            soup = BeautifulSoup(response.content, 'html.parser')
            books = extract_books_from_page(soup)
            
            for book in books:
                if book['Rating'] >= min_rating and book['Price (£)'] <= max_price:
                    genre, upc, description = extract_book_details(book['Book URL'])
                    book.update({"Genre": genre, "UPC": upc, "Description": description})
                    books_data.append(book)
            
            print(f"Página {page_num} procesada correctamente.")
            time.sleep(2)
        
        except requests.exceptions.RequestException as e:
            print(f"⚠️ Error en la página {page_num}: {e}")
            break
        
        page_num += 1
    
    return pd.DataFrame(books_data)

In [29]:
## Ejecutar y Mostrar Datos
df_books = scrape_books(min_rating=4, max_price=20)
display(df_books.head())

/tmp/ipykernel_31932/579814295.py:12: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  upc = book_soup.find('th', text='UPC').find_next_sibling('td').text.strip()


Página 1 procesada correctamente.
Página 2 procesada correctamente.
Página 3 procesada correctamente.
Página 4 procesada correctamente.
Página 5 procesada correctamente.
Página 6 procesada correctamente.
Página 7 procesada correctamente.
Página 8 procesada correctamente.
Página 9 procesada correctamente.
Página 10 procesada correctamente.
Página 11 procesada correctamente.
Página 12 procesada correctamente.
Página 13 procesada correctamente.
Página 14 procesada correctamente.
Página 15 procesada correctamente.
Página 16 procesada correctamente.
Página 17 procesada correctamente.
Página 18 procesada correctamente.
Página 19 procesada correctamente.
Página 20 procesada correctamente.
Página 21 procesada correctamente.
Página 22 procesada correctamente.
Página 23 procesada correctamente.
Página 24 procesada correctamente.
Página 25 procesada correctamente.
Página 26 procesada correctamente.
Página 27 procesada correctamente.
Página 28 procesada correctamente.
Página 29 procesada correctam

,Title,Price (£),Rating,Availability,Book URL,Genre,UPC,Description
0,Set Me Free,17.46,5,In stock,http://books.toscrape.com/catalogue/set-me-fre...,Young Adult,ce6396b0f23f6ecc,Aaron Ledbetter’s future had been planned out ...
1,The Four Agreements: A Practical Guide to Pers...,17.66,5,In stock,http://books.toscrape.com/catalogue/the-four-a...,Spirituality,6258a1f6a6dcfe50,"In The Four Agreements, don Miguel Ruiz reveal..."
2,Sophie's World,15.94,5,In stock,http://books.toscrape.com/catalogue/sophies-wo...,Philosophy,6be3beb0793a53e7,A page-turning novel that is also an explorati...
3,Untitled Collection: Sabbath Poems 2014,14.27,4,In stock,http://books.toscrape.com/catalogue/untitled-c...,Poetry,657fe5ead67a7767,"More than thirty-five years ago, when the weat..."
4,This One Summer,19.49,4,In stock,http://books.toscrape.com/catalogue/this-one-s...,Sequential Art,51653ef291ab7ddc,"Every summer, Rose goes with her mom and dad t..."


In [30]:
## Guardar los Datos en CSV
df_books.to_csv('books_scraped.csv', index=False)
print("Datos guardados en books_scraped.csv")


Datos guardados en books_scraped.csv
